In [1]:
configs_dict=dict(
#     WomenInEngineering='configs.config_WomenInEngineering_local',
    ILLAE='configs.config_ILLAE_local',
#     Juul='configs.config_Juul_local',
)

saving_dir='/Users/habibkarbasian/Documents/_Courses/_Datasets/_Retweetability/analysis/'

select_features = dict(
    static=[
        'fffs',
        'age',
        'listed',
        'verified',
        'character',
        'url',
        'media',
        'mentions',
        'reply',
#         'exclem',
#         'question',
#         'termspos',
#         'termsneg',
#         'emopos',
#         'emoneg',
#         'anew',
#         'cusswords',
#         'emojis',
        'hashtag',
        'liwc'
    ],
    dynamic=[
        # 'bow_text',
        # 'terms_text',
        'hashtags',
#         'lda_hashtags',
#         'full_text'
    ]
)

dynamic_features_files=dict(
    hashtags='_community(hashtags)_weighted_weighted_cv(all).csv',
    lda_hashtags='_lda(lda_hashtags)_1.0_1.0_0.0_20.0_cv(all).csv',
    full_text='_lda(full_text)_1.0_1.0_0.0_20.0_cv(all).csv'
)

# complete_dataset=True
complete_dataset=False

In [2]:
### init ###

import importlib

imp=importlib.import_module('_imports')
importlib.reload(imp)

util=importlib.import_module('_utils')
importlib.reload(util)

# %matplotlib inline

configs=dict()
for dataset, config in configs_dict.items():
    configs[dataset]=importlib.import_module(config)
    importlib.reload(configs[dataset])
    configs[dataset].select_features=select_features
    print('config {} is loaded.'.format(dataset))

config ILLAE is loaded.


In [3]:
### reading class labels and concatenating features and original data ###
Linear_results=imp.pd.DataFrame()
Logit_results=imp.pd.DataFrame()


for dataset_name, c in configs.items():

    dynamics=dict()
    df_inputs=dict()
    for feature in c.select_features['dynamic']:
        dynamics[c.features['dynamic'][feature]['type']]=importlib.import_module('dynamic_features.{}'.format(c.features['dynamic'][feature]['type']))
        importlib.reload(dynamics[c.features['dynamic'][feature]['type']])
        df_inputs[feature]=imp.pd.read_csv('{}data/features/{}_feature_{}({}).csv'.format(c.directory['save'],
                                                                                             c.project_name,
                                                                                             c.features['dynamic'][feature]['type'],
                                                                                             feature))
        df_inputs[feature]=dynamics[c.features['dynamic'][feature]['type']].prep_input(df_inputs[feature], c)    
    
    # dataset_name='WomenInEngineering'
    # c=configs[dataset_name]
    print(dataset_name)
    dfs=dict()

    dfs['labels'] = imp.pd.read_csv('{}data/{}_2_data_labels.csv'.format(c.directory['save'], c.project_name))
    dfs['features']=util.construct_static_features(c)

    imp.pprint('labels = {}'.format(dfs['labels'].shape))
#     util.display_df(dfs['labels'].head())

    imp.pprint('featuers = {}'.format(dfs['features'].shape))
#     util.display_df(dfs['features'].head())

    outer_ids = imp.pd.read_csv('{}{}'.format(c.directory['save'], 'cv_ids/outer_0.csv'))
    outer_ids.rename(columns={'id': c.id_name}, inplace=True)

    inner_ids = list()
    for i in range(c.inner):
      inner_ids.append(imp.pd.read_csv('{}cv_ids/inner_0_{}.csv'.format(c.directory['save'], i)))

    imp.pprint('Number of cv = {}'.format(len(inner_ids)))

    if complete_dataset:
        all_ids=imp.pd.DataFrame(dfs['features'][c.id_name])
    else:
        training = imp.copy.deepcopy(inner_ids)
        all_ids=imp.pd.DataFrame()
        for item in training:
            all_ids=imp.pd.concat([all_ids, item], ignore_index=True, axis=0)
        all_ids.rename(columns={'id':c.id_name}, inplace=True)
#         all_ids=imp.pd.concat([all_ids, outer_ids], ignore_index=True, axis=0)

    for feature in select_features['dynamic']:
        dynamic=dynamics[c.features['dynamic'][feature]['type']]
        print(c.features['dynamic'][feature]['type'])
        df_input=df_inputs[feature]
        df_dynamic = imp.pd.read_csv('{}data/features/dynamic_features/{}{}'.format(c.directory['save'],
                                                                                    c.project_name,
                                                                                    dynamic_features_files[feature]))
        cols=[c.id_name]
        for index in range(0, len(df_dynamic.columns)-1):
            cols.append('{}_{}'.format(feature, index))
        df_dynamic.columns=cols
#         util.display(df_dynamic.head())
        dfs['features']=imp.pd.merge(dfs['features'], df_dynamic, on=c.id_name, how='right')
#     util.display(df_dfs['features'].head())        


    df_normalized = util.data_prepropressing(all_ids,                                      
                                             dfs['features'],
                                             dfs['labels'],
                                             c
                                            )

    def numeric_conversion(boolean):
        return int(boolean == True)
    df_features=imp.pd.DataFrame(df_normalized['features'])
    df_labels=imp.pd.DataFrame(df_normalized['labels'].apply(numeric_conversion, axis=1))
#     util.display_df(df_features.head())   

    model = imp.LinearRegression(fit_intercept=True,
                                 normalize=True,
                                 copy_X=True,
                                 n_jobs=c.num_cores
                                )
    model.fit(df_features, df_labels)
    r_sq = model.score(df_features, df_labels)

    Linear_result=list(model.coef_)
    
    logreg = imp.LogisticRegression(fit_intercept=True,
                                    n_jobs=c.num_cores)
    logreg.fit(df_features, df_labels.values.ravel())    
    Logit_result=list(logreg.coef_)
#     LR_result.append(model.intercept_)

    Linear_results=imp.pd.concat([Linear_results, imp.pd.DataFrame(Linear_result)], axis=0)
    Logit_results=imp.pd.concat([Logit_results, imp.pd.DataFrame(Logit_result)], axis=0)


ILLAE
'labels = (17354, 2)'
'featuers = (17354, 108)'
'Number of cv = 5'
community


/Users/habibkarbasian/anaconda3/envs/LDA/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/habibkarbasian/anaconda3/envs/LDA/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 15.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


In [4]:
### Linear Regression Results ###

results=dict(
    Linear_results=Linear_results,
    Logit_results=Logit_results
)

for name, result_df in results.items():
    
    result_df.index=configs_dict.keys()
    result_df.columns=dfs['features'].columns[1:]
    # indexes=list(dfs['features'].columns)
    # del indexes[0]
    # indexes.append('cooefficient')

    # LR_results.index=indexes
    result_df=result_df.T
    result_df.to_csv('{}{}_{}.csv'.format(saving_dir,
                                          name,
                                          '_'.join(list(configs_dict.keys()))
                                         ), 
                     index=True)
    util.display_df(result_df)

,ILLAE
friends_count,0.017285
followers_count,-0.016820
favourites_count,0.022908
statuses_count,-0.045038
profile_created_at_date,-0.032946
listed_count,0.037312
verified,0.079455
text,0.065997
external_url,-0.027382
external_media_type,0.106142


,ILLAE
friends_count,0.079843
followers_count,3.348256
favourites_count,0.174640
statuses_count,-1.002950
profile_created_at_date,-0.130190
listed_count,1.449101
verified,0.352129
text,0.355985
external_url,-0.159451
external_media_type,0.489871
